<a href="https://colab.research.google.com/github/ishika2020/banana_shelf_life_prediction_model/blob/main/modeltraining.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [9]:
import os 
import matplotlib.pyplot as plt

In [10]:
len(os.listdir('/content/gdrive/MyDrive/banana dataset'))

4

In [11]:
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

In [13]:
datagen = ImageDataGenerator(
        rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.4,
        horizontal_flip=True,
        vertical_flip=False,
        fill_mode='nearest')

In [14]:
for i in range(1,7):
  os.chdir(r"/content/gdrive/MyDrive/banana dataset/train_banana/day{0}".format(i))
  print(os.getcwd())
  for path in os.listdir():
    img = load_img(f"{path}")
    x = img_to_array(img)    # this is a Numpy array with shape (3, 150, 150)
    x = x.reshape((1,) + x.shape)
    i = 0
    for batch in datagen.flow(x, batch_size=1,
                          save_to_dir=".", save_prefix='img', save_format='jpg'):
        i += 1
        if i > 10:     ## creates 10 image form 1 image 
            break 

/content/gdrive/MyDrive/banana dataset/train_banana/day1
/content/gdrive/MyDrive/banana dataset/train_banana/day2
/content/gdrive/MyDrive/banana dataset/train_banana/day3
/content/gdrive/MyDrive/banana dataset/train_banana/day4
/content/gdrive/MyDrive/banana dataset/train_banana/day5
/content/gdrive/MyDrive/banana dataset/train_banana/day6


In [15]:
for i in range(1,7):
  os.chdir(r"/content/gdrive/MyDrive/banana dataset/test_banana/day{0}".format(i))
  print(os.getcwd())
  for path in os.listdir():
    img = load_img(f"{path}")
    x = img_to_array(img)    # this is a Numpy array with shape (3, 150, 150)
    x = x.reshape((1,) + x.shape)
    i = 0
    for batch in datagen.flow(x, batch_size=1,
                          save_to_dir=".", save_prefix='img', save_format='jpg'):
        i += 1
        if i > 10:     ## creates 10 image form 1 image 
            break 

/content/gdrive/MyDrive/banana dataset/test_banana/day1
/content/gdrive/MyDrive/banana dataset/test_banana/day2
/content/gdrive/MyDrive/banana dataset/test_banana/day3
/content/gdrive/MyDrive/banana dataset/test_banana/day4
/content/gdrive/MyDrive/banana dataset/test_banana/day5
/content/gdrive/MyDrive/banana dataset/test_banana/day6


**Model Building**

In [16]:
from keras.layers import Input,Lambda,Dense,Flatten
from keras.models import Model 
from keras.applications.vgg16 import VGG16 
from keras.applications.vgg16 import preprocess_input
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
import numpy as np

In [17]:
IMAGE_SIZE = [224, 224]

In [18]:
vgg = VGG16(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)

58900480/58889256 [==============================] - 0s 0us/step


In [19]:
for layer in vgg.layers:  
  layer.trainable = False

In [20]:
x = Flatten()(vgg.output)
prediction = Dense(6, activation='softmax')(x)
model = Model(inputs=vgg.input, outputs=prediction)

In [21]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [22]:
model.compile( loss='categorical_crossentropy',  
               optimizer='adam',  
               metrics=['accuracy'])

**the training set and test set**

In [29]:
train_datagen = ImageDataGenerator(rescale = 1./255,                          
                                    shear_range = 0.2,
                                   zoom_range = 0.2,
                                    horizontal_flip= True)
training_set = train_datagen.flow_from_directory('/content/gdrive/MyDrive/banana dataset/train_banana',
                                           target_size = (224, 224),
                                             batch_size = 16,                             
                                        class_mode = 'categorical')

Found 2207 images belonging to 6 classes.


In [30]:
test_datagen = ImageDataGenerator(rescale = 1./255)
test_set = test_datagen.flow_from_directory('/content/gdrive/MyDrive/banana dataset/test_banana',
                                         target_size = (224, 224),
                                         batch_size=16,
                                         class_mode = 'categorical')

Found 681 images belonging to 6 classes.


In [31]:
r = model.fit_generator(training_set,  validation_data=test_set,  epochs=10,steps_per_epoch=len(training_set),validation_steps=len(test_set))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  """Entry point for launching an IPython kernel.


Epoch 1/10
138/138 [==============================] - 1451s 11s/step - loss: 0.3413 - accuracy: 0.8749 - val_loss: 2.7952 - val_accuracy: 0.4508
Epoch 2/10
138/138 [==============================] - 1448s 11s/step - loss: 0.2401 - accuracy: 0.9126 - val_loss: 2.8586 - val_accuracy: 0.4758
Epoch 3/10
138/138 [==============================] - 1439s 10s/step - loss: 0.1753 - accuracy: 0.9343 - val_loss: 2.9313 - val_accuracy: 0.4846
Epoch 4/10
138/138 [==============================] - 1439s 10s/step - loss: 0.1791 - accuracy: 0.9325 - val_loss: 2.9204 - val_accuracy: 0.4846
Epoch 5/10
138/138 [==============================] - 1458s 11s/step - loss: 0.1775 - accuracy: 0.9329 - val_loss: 3.3826 - val_accuracy: 0.4802
Epoch 6/10
138/138 [==============================] - 1444s 10s/step - loss: 0.1741 - accuracy: 0.9366 - val_loss: 3.3374 - val_accuracy: 0.4655
Epoch 7/10
138/138 [==============================] - 1452s 11s/step - loss: 0.1849 - accuracy: 0.9384 - val_loss: 3.5193 - val_ac

In [32]:
model.save("banana_shelf_life1.h5")

In [33]:
import matplotlib.pyplot as plt
